In [131]:
reset -fs

In [132]:
import pandas as pd
import numpy as np
import re, string

import nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
  
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import gensim
from gensim import corpora

from collections import Counter
import string

from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer

import zipfile
import os

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mariavasilenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariavasilenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Data pre-processing

- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- Words that have fewer than 3 characters are removed.
- All stopwords are removed.
 - Words are __lemmatized__ — words in third person are changed to first person and verbs in past and future tenses are changed into present.
- Words are __stemmed__ — words are reduced to their root form.


In [269]:
# Create the list of stopwords and extend it 

stopwords = list(ENGLISH_STOP_WORDS)
# Adding "just" after running basic LDA model
newStopWords = ['just', ""]
stopwords.extend(newStopWords)


def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text.
    Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3.
    """
    
    text=text.lower() #Convert everything to lowercase
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words =  nltk.word_tokenize(nopunct) # tokenizing words
    words = [w for w in words if len(w) > 3]  # Drop words less than length 3
    words[:] = [w for w in words if w != '']
    words = [w for w in words if w not in stopwords] # Removes stop words using SciKit-Learn's ENGLISH_STOP_WORDS set.
    words = [w.encode('ascii','ignore') for w in words]
    return words

def stemwords(words):
    """
    Given a list of tokens/words, return a new list with each word
    stemmed using a PorterStemmer.
    """
    stemmer = PorterStemmer()
    words = [w.decode('ascii','ignore') for w in words]
    stemmed = [stemmer.stem(w) for w in words]
    return stemmed

def lemmatize_stemming(words):
    words = [w.decode('ascii','ignore') for w in words]
    text = " ".join(words)
    stemmer = PorterStemmer()
    stemmed = stemmer.stem(WordNetLemmatizer().lemmatize(text))
    return stemmed.split()

def lemmatize (tokens):
    '''
    Given a list of tokens/words, return a list of lemmatized words 
    i.e. words in third person are changed to first person and verbs in past,
    and future tenses are changed into present.
    '''
    text = " ".join(tokens)
    lemmatizer = WordNetLemmatizer()
    
    return lemmatizer.lemmatize(text).split()
    
    
def tokenizer(text):
    
    return stemwords(tokenize(text))


In [273]:
df_reviews = pd.read_csv('data/appstore_googleplay_all_reviews.csv')

In [274]:
df_reviews.tail()

,name,id,title,author_name,author_uri,voteSum,voteCount,rating,text,date
6113,Google Fit: Health and Activity Tracking,com.google.android.apps.fitness,NaN,Mohit G,107526919134556209809,NaN,NaN,1,This app does not work property on my one plus...,2018-11-18 15:03:20
6114,Google Fit: Health and Activity Tracking,com.google.android.apps.fitness,NaN,sagar suryawanshi,101956979010176828886,NaN,NaN,1,I want to add leadboard How?,2018-11-18 14:54:32
6115,Medical Records App,com.cliniconline,NaN,Abod Osama,115080537418668508998,NaN,NaN,4,A great useful application but how can I trans...,2018-11-19 06:49:38
6116,Medical Records App,com.cliniconline,NaN,Agnes Novita,110614731832618616830,NaN,NaN,5,"Simple, easy, and useful",2018-11-18 03:48:23
6117,Medical Records App,com.cliniconline,NaN,Alfredo Baruc,116943082422817884463,NaN,NaN,5,Very easy to use and is what I am looking for ...,2018-11-13 05:14:25


In [275]:
len(df_reviews)

6118

In [276]:
# Create a dataframe with negative reviews
df_neg_reviews = df_reviews[df_reviews['rating']<=3]

In [277]:
#Create dataframe with positive reviews
df_pos_reviews = df_reviews[df_reviews['rating']>3]

In [278]:
len(df_neg_reviews)

3735

In [279]:
len(df_pos_reviews)

2383

In [280]:
assert len(df_neg_reviews) + len(df_pos_reviews) == len(df_reviews)

In [281]:
# Extract text of negative reviews
neg_reviews_text = df_neg_reviews['text']
neg_text = list(neg_reviews_text)

In [282]:
# Extract the text of positive reviews
pos_reviews_text = df_pos_reviews['text']
pos_text = list(pos_reviews_text)

In [283]:
neg_clean_text = [tokenizer(n_t) for n_t in neg_text]

In [284]:
len(neg_clean_text)

3735

In [285]:
pos_clean_text = [tokenizer(p_t) for p_t in pos_text]

In [286]:
len(pos_clean_text)

2383

In [287]:
# Adding bigrams and trigrams to negative reviews

from gensim.models.phrases import Phrases, Phraser
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phraser(Phrases(neg_clean_text, min_count=10))
trigram = Phrases(bigram[neg_clean_text])

for idx in range(len(neg_clean_text)):
    for token in bigram[neg_clean_text[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            neg_clean_text[idx].append(token)
    for token in trigram[neg_clean_text[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            neg_clean_text[idx].append(token)

/anaconda3/envs/uhc_env/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [288]:
# Adding bigrams and trigrams to positive reviews

from gensim.models.phrases import Phrases, Phraser
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phraser(Phrases(pos_clean_text, min_count=10))
trigram = Phrases(bigram[pos_clean_text])

for idx in range(len(pos_clean_text)):
    for token in bigram[pos_clean_text[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            pos_clean_text[idx].append(token)
    for token in trigram[pos_clean_text[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            pos_clean_text[idx].append(token)

In [ ]:
# Remove empty tokens

In [302]:
for item in neg_clean_text:
    item[:] = [elem for elem in item if elem != ""]


In [305]:
for item in pos_clean_text:
    item[:] = [elem for elem in item if elem != ""]


In [308]:
#Creating the dictionary out of corpus of negative reviews
neg_dictionary = corpora.Dictionary(neg_clean_text)

In [309]:
# Creating the dictionary out of corpus of positive reviews
pos_dictionary = corpora.Dictionary(pos_clean_text)

In [306]:
for idx, item in enumerate(pos_clean_text):
        if '' in item:
            print(idx, "YES")


In [310]:
len(neg_dictionary)

3867

In [311]:
len(pos_dictionary)

2845

In [312]:
# Hyperparameters

NUM_NEG_TOPICS =5 # Number of topics in negative reviews
NUM_POS_TOPICS =5 # Number of topics in positive reviews
# PArameters used in filtering the dictionaries
MIN_DF= 2 # remove tokens that appear in less than MIN_DF docs
MAX_DF = 0.5 # Remove tokens that appear in more than MAX_DF * 100% of docs 

In [313]:
# Filter extremes:
# Filter tokens that appear in less than 2 doc-s
# Filter out tokens that appear in more than 30% of docs

neg_dictionary.filter_extremes(no_below=MIN_DF, no_above= MAX_DF)
pos_dictionary.filter_extremes(no_below=MIN_DF, no_above= MAX_DF)


In [314]:
len(neg_dictionary)

2298

In [315]:
len(pos_dictionary)

1633

In [316]:
neg_doc_term_mx = [neg_dictionary.doc2bow(doc) for doc in neg_clean_text]

In [317]:
pos_doc_term_mx = [pos_dictionary.doc2bow(doc) for doc in pos_clean_text]

In [318]:
# LDA for negative reviews

neg_lda = gensim.models.ldamodel.LdaModel(corpus=neg_doc_term_mx, \
                                      id2word=neg_dictionary, num_topics=NUM_NEG_TOPICS, random_state = 17) \
                                      #, update_every=1, chunksize=100, passes=50)

In [319]:
# LDA for positive reviews

pos_lda = gensim.models.ldamodel.LdaModel(corpus=pos_doc_term_mx, \
                                      id2word=pos_dictionary, num_topics=NUM_POS_TOPICS, random_state = 17) \
                                      #, update_every=1, chunksize=100, passes=50)

In [320]:
# Topics based on negative reviews
neg_lda.print_topics(num_words=20)

[(0,
  '0.017*"card" + 0.016*"insur_card" + 0.014*"updat" + 0.013*"time" + 0.012*"work" + 0.010*"inform" + 0.009*"doe" + 0.009*"tri" + 0.009*"insur" + 0.009*"use" + 0.008*"crash" + 0.008*"version" + 0.007*"phone" + 0.007*"like" + 0.006*"custom_servic" + 0.006*"provid" + 0.006*"abl" + 0.006*"white_screen" + 0.006*"doctor" + 0.006*"iphon"'),
 (1,
  '0.022*"doesn_work" + 0.019*"work" + 0.014*"doesn" + 0.011*"need" + 0.010*"time" + 0.010*"messag" + 0.009*"like" + 0.009*"updat" + 0.008*"email" + 0.008*"make" + 0.007*"use" + 0.007*"card" + 0.007*"doctor" + 0.007*"version" + 0.007*"custom_servic" + 0.006*"data" + 0.006*"test_result" + 0.006*"user" + 0.006*"login" + 0.006*"want"'),
 (2,
  '0.029*"work" + 0.012*"urgent_care" + 0.012*"doctor" + 0.011*"doesn_work" + 0.011*"password" + 0.009*"doesn" + 0.009*"time" + 0.009*"search" + 0.009*"updat" + 0.009*"error_messag" + 0.009*"use" + 0.009*"doesnt_work" + 0.009*"health_insur" + 0.008*"login" + 0.007*"past_password" + 0.007*"useless" + 0.007*"wast

Negative topics:

- Topic 0: insurance card,  ==> TECHNICAL ISSUES: bad CUSTOMER SERVICE, ISSUES WITH PASSWORD
- Topic 1: Issues with messages, e-mails to doctor ==> MESSAGING TO DOCTOR
- Topic 2: URGENT CARE
- Topic 3: APPOINTMENT SCHEDULING
- Topic 4:  issues with LOGIN/PASSWORD

In [321]:
#Visualize negative topics
pyLDAvis.gensim.prepare(neg_lda, neg_doc_term_mx, neg_dictionary, R = 20)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.002552  0.021983       1        1  27.573856
1      0.013450  0.001616       2        1  23.328445
3     -0.031702 -0.086878       3        1  17.654156
0      0.095862  0.011876       4        1  16.556755
2     -0.075058  0.051403       5        1  14.886790, topic_info=     Category         Freq             Term        Total  loglift  logprob
term                                                                      
642   Default   221.000000       insur_card   221.000000  20.0000  20.0000
289   Default   574.000000       doesn_work   574.000000  19.0000  19.0000
1596  Default   129.000000      urgent_care   129.000000  18.0000  18.0000
1088  Default   141.000000     make_appoint   141.000000  17.0000  17.0000
1425  Default   266.000000     error_messag   266.000000  16.0000  16.0000
275   Default   510.000000         password   510.000000  15.0000  15.0000
1379  Default   125.000000      doesnt_work   125.000000  14.0000  14.0000
448   Default   109.000000     latest_updat   109.000000  13.0000  13.0000
69    Default   267.000000          appoint   267.000000  12.0000  12.0000
1341  Default   126.000000     health_insur   126.000000  11.0000  11.0000
1483  Default   148.000000    past_password   148.000000  10.0000  10.0000
67    Default   583.000000            updat   583.000000   9.0000   9.0000
461   Default   352.000000        wast_time   352.000000   8.0000   8.0000
40    Default  1196.000000             work  1196.000000   7.0000   7.0000
637   Default   473.000000             card   473.000000   6.0000   6.0000
1140  Default   206.000000           search   206.000000   5.0000   5.0000
1517  Default    71.000000     blank_screen    71.000000   4.0000   4.0000
192   Default   225.000000          version   225.000000   3.0000   3.0000
891   Default    94.000000     white_screen    94.000000   2.0000   2.0000
1508  Default   125.000000      send_messag   125.000000   1.0000   1.0000
137    Topic1    55.852753    creat_account    57.218452   1.2641  -5.7614
1414   Topic1    16.200756           ticket    17.061787   1.2365  -6.9991
869    Topic1    32.284027  answer_question    34.179459   1.2312  -6.3096
1492   Topic1    16.016876          couldnt    17.058525   1.2253  -7.0105
582    Topic1     9.270458          protect    10.083233   1.2043  -7.5573
1827   Topic1     9.120960             desk     9.975293   1.1988  -7.5736
1364   Topic1     8.283414       keep_close     9.064422   1.1982  -7.6699
2029   Topic1     7.717679  softwar_develop     8.556878   1.1851  -7.7406
467    Topic1     6.348033             wise     7.138539   1.1709  -7.9360
1525   Topic1     6.317826           obviou     7.138985   1.1661  -7.9408
...       ...          ...              ...          ...      ...      ...
2077   Topic5     7.186662            treat     8.677637   1.7162  -7.1955
1825   Topic5     3.587720          starter     4.369410   1.7076  -7.8902
517    Topic5    59.837852       keep_crash    73.428795   1.7000  -5.0761
125    Topic5    46.506199    work_properli    57.205215   1.6976  -5.3282
1456   Topic5    21.784849    search_doctor    27.669693   1.6656  -6.0865
209    Topic5    32.349304       wast_space    42.397667   1.6342  -5.6912
819    Topic5    49.788754     recent_updat    68.349770   1.5878  -5.2600
1379   Topic5    83.850266      doesnt_work   125.600311   1.5006  -4.7387
1341   Topic5    82.406044     health_insur   126.232040   1.4782  -4.7561
363    Topic5    50.772610  complet_useless    76.391586   1.4962  -5.2404
1365   Topic5    32.536087        zero_star    47.763729   1.5208  -5.6854
1482   Topic5    45.415791   password_manag    82.397308   1.3090  -5.3519
1483   Topic5    69.062645    past_password   148.655365   1.1381  -4.9327
1140   Topic5    88.032402           search   206.581879   1.0517  -4.6900
508    Topic5    63.894794        work_fine   148.3998

- Topic 1 in pyLDA vis corresponds to Topic 4 in lda print results 
- Topic 2 in pyLDA vis corresponds to Topic 1 in lda print results 
- Topic 3 in pyLDA vis corresponds to Topic 3 in lda print results 
- Topic 4 in pyLDA vis corresponds to Topic 0 in lda print results 
- Topic 5 in pyLDA vis corresponds to Topic 2 in lda print results 

The left panel of the chart presents a global view of topics: how they are related to each other (far apart or inmtersect) and how preevalent they are (the larger the circle the more prevalent is the topic). 

THe right panel shows  the individual terms that are the most useful for interpreting the currently selected topic on the left.  A pair of overlaid bars represent both the corpus-wide frequency ofa given term as well as the topic-specific frequency of the term.

Parameter lambda regulates the _relevance_ of the term, which is used to rank terms within topics. It is regulated by parameter _lambda_.
- If _lambda_ = 1, we'll see familiar ranking of terms in decreasing order of their topic-specific probability
- If _lambda_ = 0, we'll rank words soleley by their _lift_, defined as the ratio of a term’s probability within a topic to its marginal probability across the corpus. In other words, it helps determine the words exclusivity to a particular topic. 

In [324]:
# Topics based on positive reviews
pos_lda.print_topics()

[(0,
  '0.030*"great" + 0.018*"use" + 0.018*"doctor" + 0.018*"need" + 0.017*"test_result" + 0.016*"like" + 0.014*"love" + 0.013*"medic" + 0.013*"inform" + 0.013*"info"'),
 (1,
  '0.020*"user_friendli" + 0.014*"doctor" + 0.013*"great" + 0.011*"like" + 0.011*"inform" + 0.011*"love" + 0.011*"track" + 0.011*"abl" + 0.010*"record" + 0.010*"use"'),
 (2,
  '0.034*"love" + 0.027*"good" + 0.017*"work" + 0.017*"need" + 0.015*"nice" + 0.013*"inform" + 0.013*"like" + 0.013*"great" + 0.013*"easi" + 0.012*"conveni"'),
 (3,
  '0.030*"test_result" + 0.018*"great" + 0.018*"appoint" + 0.018*"doctor" + 0.016*"use" + 0.015*"medic" + 0.015*"result" + 0.014*"make" + 0.014*"help" + 0.013*"time"'),
 (4,
  '0.051*"easi" + 0.027*"medic" + 0.022*"medic_record" + 0.019*"easi_navig" + 0.016*"great" + 0.014*"doctor" + 0.013*"medic_histori" + 0.013*"love" + 0.011*"appoint" + 0.011*"time"')]

__Suggested topics based on positive reviews__
- Topic 0: great, test results ==> FEATURE: access to  TEST RESULTS
- Topic 1: user friendly, great ==> good UX DESIGN
FEATURE: access to MEDICAL RECORDS
- Topic 2: Love, easy to use ==> 
- Topic 3: test results, doctor appointments ==> DOCTOR APPOINTMENTS 
- Topic 4: easy to navigate, medical records ==> access to MEDICAL RECORDS


In [323]:
#Visualize positive topics
pyLDAvis.gensim.prepare(pos_lda, pos_doc_term_mx, pos_dictionary, R = 20)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000087 -0.035587       1        1  24.814779
2     -0.059981  0.050770       2        1  22.068804
1     -0.061460 -0.006336       3        1  20.097555
4      0.083978  0.064292       4        1  16.769739
3      0.037550 -0.073139       5        1  16.249132, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
311   Default  503.000000             easi  503.000000  20.0000  20.0000
623   Default  330.000000      test_result  330.000000  19.0000  19.0000
1208  Default  135.000000       easi_navig  135.000000  18.0000  18.0000
425   Default  211.000000    user_friendli  211.000000  17.0000  17.0000
12    Default  249.000000     medic_record  249.000000  16.0000  16.0000
419   Default  278.000000          appoint  278.000000  15.0000  15.0000
438   Default  108.000000    medic_histori  108.000000  14.0000  14.0000
168   Default  376.000000             good  376.000000  13.0000  13.0000
954   Default   88.000000     email_doctor   88.000000  12.0000  12.0000
11    Default  464.000000            medic  464.000000  11.0000  11.0000
370   Default   67.000000        save_time   67.000000  10.0000  10.0000
143   Default  428.000000           doctor  428.000000   9.0000   9.0000
224   Default  150.000000          conveni  150.000000   8.0000   8.0000
612   Default   85.000000       keep_track   85.000000   7.0000   7.0000
686   Default   74.000000       finger_tip   74.000000   6.0000   6.0000
352   Default  161.000000             test  161.000000   5.0000   5.0000
128   Default  237.000000           result  237.000000   4.0000   4.0000
335   Default  175.000000             nice  175.000000   3.0000   3.0000
46    Default  562.000000             love  562.000000   2.0000   2.0000
1362  Default   67.000000  refil_prescript   67.000000   1.0000   1.0000
1335   Topic1   27.114597            drill   28.884371   1.3305  -5.7122
1184   Topic1   16.727005           scroll   18.280655   1.3049  -6.1953
1018   Topic1   17.757336             alot   19.641396   1.2929  -6.1355
1295   Topic1   16.944311            sleep   19.650896   1.2455  -6.1824
924    Topic1    6.313101           assess    7.336459   1.2435  -7.1697
1154   Topic1    5.912184             wear    6.920513   1.2363  -7.2353
1012   Topic1    5.911130             puls    6.920563   1.2361  -7.2355
1567   Topic1    5.903206            arrow    6.919875   1.2348  -7.2368
902    Topic1    5.877542          discuss    6.917938   1.2308  -7.2411
1079   Topic1    5.868512            chanc    6.918736   1.2291  -7.2427
...       ...         ...              ...         ...      ...      ...
947    Topic5   10.440621              doc   14.293973   1.5030  -6.2432
594    Topic5   35.911900  schedul_appoint   49.402313   1.4982  -5.0078
954    Topic5   60.118740     email_doctor   88.653107   1.4287  -4.4926
1583   Topic5   13.324775           clinic   19.039558   1.4602  -5.9993
310    Topic5   51.789406           commun   90.996475   1.2535  -4.6417
623    Topic5  158.841049      test_result  330.937592   1.0831  -3.5210
367    Topic5   21.929356          schedul   34.757229   1.3566  -5.5011
112    Topic5   13.070212             bodi   19.279011   1.4284  -6.0186
1123   Topic5   12.189662            upcom   17.858114   1.4353  -6.0883
612    Topic5   42.582104       keep_track   85.775864   1.1168  -4.8375
277    Topic5   18.051447         diagnosi   29.812382   1.3154  -5.6957
352    Topic5   64.583488             test  161.823303   0.8986  -4.4209
419    Topic5   96.140244          appoint  278.353333   0.7540  -4.0231
772    Topic5   35.840240            refil   79.920929   1.0152  -5.0098
128    Topic5   78.336395           result  237.707596   0.7071  -4.2279
1362   Topic5   30.976454  refil_prescript   67.866653   1.0328  -5.1557
56

- Topic 1 in pyLDA vis corresponds to Topic 0 in lda print results 
- Topic 2 in pyLDA vis corresponds to Topic 2 in lda print results 
- Topic 3 in pyLDA vis corresponds to Topic 1 in lda print results 
- Topic 4 in pyLDA vis corresponds to Topic 4 in lda print results 
- Topic 5 in pyLDA vis corresponds to Topic 3 in lda print results 
